In [1]:
import typing as T  # pylint: disable=unused-import

import numpy as np
import numpy.typing as npt

from pydrake.solvers import (  # pylint: disable=import-error, no-name-in-module, unused-import
    MathematicalProgram,
    MathematicalProgramResult,
    Solve,
    MosekSolver,
    GurobiSolver,
    MosekSolverDetails,
    SnoptSolver,
    OsqpSolver,
    ClarabelSolver,
    IpoptSolver,
    SolverOptions,
    CommonSolverOption,
)
from pydrake.geometry.optimization import (  # pylint: disable=import-error, no-name-in-module
    GraphOfConvexSets,
    GraphOfConvexSetsOptions,
    HPolyhedron,
    Point,
    ConvexSet,
    Hyperrectangle,
)
import numbers
import pydot

from pydrake.symbolic import (  # pylint: disable=import-error, no-name-in-module, unused-import
    Polynomial,
    Variable,
    Variables,
    Expression,
)
from pydrake.math import (  # pylint: disable=import-error, no-name-in-module, unused-import
    ge,
    eq,
    le,
)


In [4]:
gcs = GraphOfConvexSets()

# generate 100 random square sets:
np.random.seed = 1
for i in range(500):
    lb = np.random.uniform(0,100, 2)
    wh = np.random.uniform(10,20, 2)
    v = gcs.AddVertex(Hyperrectangle(lb, lb + wh))
start = gcs.AddVertex(Hyperrectangle(np.zeros(2), np.ones(2)*10))
target = gcs.AddVertex(Hyperrectangle(np.ones(2)*90, np.ones(2)*100))
vertices = gcs.Vertices()
for i, v in enumerate(vertices):
    for j in range(i+1, len(vertices)):
        w = vertices[j]
        if not v.set().IntersectsWith(w.set()):
            e = gcs.AddEdge(v, w)
            e.AddCost( np.sum([(e.xu()[i]-e.xv()[i])**2 for i in range(2)]) )


In [6]:
gcs_options = GraphOfConvexSetsOptions()
gcs_options.convex_relaxation = True
gcs_options.max_rounding_trials = 100
gcs_options.max_rounded_paths = 100
gcs_options.solver = MosekSolver()

result = gcs.SolveShortestPath(start, target, options = gcs_options)

print("Solved: ", result.is_success())
print("Time: ", result.get_solver_details().optimizer_time)

INFO:drake:Solved GCS shortest path using Mosek with convex_relaxation=true and preprocessing=false and rounding.
INFO:drake:Finished 100 rounding trials.


Solved:  True
Time:  0.0002968311309814453
